# Dask tutorial

In [1]:
# PyArrow is a dependency of Dask
# !sudo apt-get update
# !sudo apt-get install python3-dask
# !pip3 install pyarrow

import dask
import dask.dataframe as dd

# When dask uses pandas 2.0.0+ it casts object columns to string automatically ([("i", 0.48)] -> '[("i", 0.48)]')
_ = dask.config.set({"dataframe.convert-string": False})

Dask Dataframes are just collections of Pandas-like Dataframes.

In [2]:
dataset_path = "smadex-challenge-predict-the-revenue/train/train"
filters = [("datetime", ">=", "2025-10-01-00-00"), ("datetime", "<", "2025-10-13-00-00")]

ddf = dd.read_parquet(
    dataset_path,
    filters = filters
)

We can operate with Dask Dataframes with the same API as the Pandas DataFrames...

In [3]:
ddf.head()

,buyer_d1,buyer_d7,buyer_d14,buyer_d28,buy_d7,buy_d14,buy_d28,iap_revenue_d7,iap_revenue_d14,iap_revenue_d28,...,user_bundles_l28d,weekend_ratio,weeks_since_first_seen,wifi_ratio,whale_users_bundle_num_buys_prank,whale_users_bundle_revenue_prank,whale_users_bundle_total_num_buys,whale_users_bundle_total_revenue,row_id,datetime
0,0,1,1,1,1,1,1,2.147718,2.147718,2.147718,...,"[88981729bd5c1e5aea9ada4bce00a2531e9e98f7, 25c...",0.019802,6.0,0.913366,None,None,None,None,819ecc0e-1a97-43ed-83f6-b9ede4f7fc48,2025-10-01-00-00
1,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,None,NaN,NaN,NaN,None,None,None,None,0a7fbf18-5041-42af-bd0a-0cb6586b8598,2025-10-01-00-00
2,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[6506b7e0a24666debd08f74266800f2eb154df5a, 150...",0.399021,6.0,0.999388,None,None,None,None,fc1a2689-b136-4ffa-b23b-9d8215bd720f,2025-10-01-00-00
3,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[2b472e3dc96f1847490d7411b25e12ed417b9714, 3ba...",0.121547,6.0,1.000000,None,None,None,None,0340fcc6-50bd-42ab-b9f4-4c1184b640cb,2025-10-01-00-00
4,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[1031535cf2a1315422fd05d321349bcd3c3ffc04, 478...",0.293285,6.0,0.160243,None,None,None,None,219d253f-bef4-4039-84b2-ed55f009cc43,2025-10-01-00-00


...but we cannot actually use them like this. Note that getting the header of the dataframe had a lot of computational cost because we had to load the data into memory.

In [4]:
ddf

,buyer_d1,buyer_d7,buyer_d14,buyer_d28,buy_d7,buy_d14,buy_d28,iap_revenue_d7,iap_revenue_d14,iap_revenue_d28,registration,retention_d1_to_d7,retention_d3_to_d7,retention_d7_to_d14,retention_d1,retention_d3,retentiond7,advertiser_bundle,advertiser_category,advertiser_subcategory,advertiser_bottom_taxonomy_level,carrier,country,region,dev_make,dev_model,dev_os,dev_osv,hour,release_date,release_msrp,weekday,avg_act_days,avg_daily_sessions,avg_days_ins,avg_duration,bcat,bcat_bottom_taxonomy,bundles_cat,bundles_cat_bottom_taxonomy,bundles_ins,city_hist,country_hist,cpm,cpm_pct_rk,ctr,ctr_pct_rk,dev_language_hist,dev_osv_hist,first_request_ts,first_request_ts_bundle,first_request_ts_category_bottom_taxonomy,hour_ratio,iap_revenue_usd_bundle,iap_revenue_usd_category,iap_revenue_usd_category_bottom_taxonomy,last_buy,last_buy_ts_bundle,last_buy_ts_category,last_ins,last_install_ts_bundle,last_install_ts_category,advertiser_actions_action_count,advertiser_actions_action_last_timestamp,user_actions_bundles_action_count,user_actions_bundles_action_last_timestamp,last_advertiser_action,new_bundles,num_buys_bundle,num_buys_category,num_buys_category_bottom_taxonomy,region_hist,rev_by_adv,rwd_prank,user_bundles,user_bundles_l28d,weekend_ratio,weeks_since_first_seen,wifi_ratio,whale_users_bundle_num_buys_prank,whale_users_bundle_revenue_prank,whale_users_bundle_total_num_buys,whale_users_bundle_total_revenue,row_id,datetime
npartitions=144,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int32,int32,int32,int32,int64,int64,int64,float64,float64,float64,int32,int32,int32,int32,int32,int32,int32,object,object,object,object,object,object,object,object,object,object,object,object,object,int64,int32,float64,object,float64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,int64,object,object,object,object,object,object,int64,object,object,int64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float64,int32,float64,object,object,object,object,object,category[known]
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


We need to manually get hold of the partitions individually. We can do this easily by converting the DataFrame into a list of Dask Delayed objects.

In [5]:
parts = [part for part in ddf.to_delayed()]

In [6]:
type(parts[0])

dask.delayed.Delayed

Then load each part separately. Just loading one partition is not _too_ slow.

In [7]:
part_0 = parts[0].compute()
part_0.head()

,buyer_d1,buyer_d7,buyer_d14,buyer_d28,buy_d7,buy_d14,buy_d28,iap_revenue_d7,iap_revenue_d14,iap_revenue_d28,...,user_bundles_l28d,weekend_ratio,weeks_since_first_seen,wifi_ratio,whale_users_bundle_num_buys_prank,whale_users_bundle_revenue_prank,whale_users_bundle_total_num_buys,whale_users_bundle_total_revenue,row_id,datetime
0,0,1,1,1,1,1,1,2.147718,2.147718,2.147718,...,"[88981729bd5c1e5aea9ada4bce00a2531e9e98f7, 25c...",0.019802,6.0,0.913366,None,None,None,None,819ecc0e-1a97-43ed-83f6-b9ede4f7fc48,2025-10-01-00-00
1,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,None,NaN,NaN,NaN,None,None,None,None,0a7fbf18-5041-42af-bd0a-0cb6586b8598,2025-10-01-00-00
2,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[6506b7e0a24666debd08f74266800f2eb154df5a, 150...",0.399021,6.0,0.999388,None,None,None,None,fc1a2689-b136-4ffa-b23b-9d8215bd720f,2025-10-01-00-00
3,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[2b472e3dc96f1847490d7411b25e12ed417b9714, 3ba...",0.121547,6.0,1.000000,None,None,None,None,0340fcc6-50bd-42ab-b9f4-4c1184b640cb,2025-10-01-00-00
4,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[1031535cf2a1315422fd05d321349bcd3c3ffc04, 478...",0.293285,6.0,0.160243,None,None,None,None,219d253f-bef4-4039-84b2-ed55f009cc43,2025-10-01-00-00


The loaded partition is a Pandas DataFrame.

In [8]:
type(part_0)

pandas.core.frame.DataFrame

In [9]:
part_0.describe()

,buyer_d1,buyer_d7,buyer_d14,buyer_d28,buy_d7,buy_d14,buy_d28,iap_revenue_d7,iap_revenue_d14,iap_revenue_d28,...,release_msrp,weekday,avg_act_days,avg_days_ins,first_request_ts,last_buy,last_ins,weekend_ratio,weeks_since_first_seen,wifi_ratio
count,121887.000000,121887.000000,121887.000000,121887.000000,121887.000000,121887.000000,121887.000000,121887.000000,121887.000000,121887.000000,...,109905.000000,121887.0,61107.000000,7183.000000,5.390700e+04,2.374000e+03,2.052200e+04,63305.000000,67489.000000,63304.000000
mean,0.031217,0.042301,0.044615,0.046601,0.095096,0.124853,0.174161,8.001834,8.308581,9.071367,...,564.108494,3.0,3.998519,5.228739,1.758773e+09,1.758279e+09,1.758424e+09,0.329158,4.877195,0.641166
std,0.173906,0.201277,0.206458,0.210783,0.996749,1.397024,2.388029,2048.960358,2049.186514,2050.423635,...,514.951762,0.0,2.115821,5.584633,3.757641e+05,7.122233e+05,7.406386e+05,0.270035,1.947677,0.398610
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,30.000000,3.0,1.000000,0.000000,1.758067e+09,1.756600e+09,1.756771e+09,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,180.000000,3.0,2.000000,1.000000,1.758499e+09,1.757735e+09,1.757857e+09,0.129870,4.000000,0.221200
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,299.000000,3.0,4.000000,3.333333,1.758499e+09,1.758478e+09,1.758611e+09,0.279810,6.000000,0.842471
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,899.000000,3.0,6.000000,7.000000,1.759190e+09,1.758888e+09,1.759091e+09,0.458333,6.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,136.000000,150.000000,368.000000,696622.140000,696622.140000,696622.140000,...,5160.000000,3.0,7.000000,28.000000,1.759190e+09,1.759104e+09,1.759277e+09,1.000000,6.000000,1.000000


In [10]:
part_0.isna()

,buyer_d1,buyer_d7,buyer_d14,buyer_d28,buy_d7,buy_d14,buy_d28,iap_revenue_d7,iap_revenue_d14,iap_revenue_d28,...,user_bundles_l28d,weekend_ratio,weeks_since_first_seen,wifi_ratio,whale_users_bundle_num_buys_prank,whale_users_bundle_revenue_prank,whale_users_bundle_total_num_buys,whale_users_bundle_total_revenue,row_id,datetime
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,True,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,True,True,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,True,True,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121882,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,False,False
121883,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,True,True,False,False
121884,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,False,False
121885,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,True,True,False,False


## Using Dask with PyTorch

The main objective is to be able to use PyTorch's DataLoaders with our Dask DataFrame. This is difficult because PyTorch does not accept Dask DataFrames.

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils as utils

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

The _state-of-the-art_ solution (ChatGPT's suggestion) is to create a subclass of the IterableDataset.

In [68]:
import random

class DaskIterableDataset(utils.data.IterableDataset):
    """A PyTorch IterableDataset fitted to accept Dask DataFrames as input."""
    
    def __init__ (self, ddf: dd.DataFrame,
                  batch_size: int = 1024):
        self.ddf = ddf
        self.bs = batch_size

    def __iter__ (self):
        ddf.shuffle()
        for part in self.ddf.to_delayed():
            pdf = part.compute()
            pdf.sample(frac = 1) # Randomize, no replacement
            for i in range(int(pdf.shape[0] / self.bs)):
                yield pdf.iloc[i * self.bs : min((i + 1) * self.bs, pdf.shape[0]), :]
            del pdf   # optional but safe
            break     # Remove when you are going to use it later

Then iterate over it.

In [13]:
dataset = DaskIterableDataset(ddf)

In [69]:
import time
t = time.time()

for data in dataset:
    print(data)

print(time.time() - t)

      buyer_d1  buyer_d7  buyer_d14  buyer_d28  buy_d7  buy_d14  buy_d28  \
0            0         0          0          0       0        0        0   
1            0         0          0          0       0        0        0   
2            0         0          0          0       0        0        0   
3            0         0          0          0       0        0        0   
4            0         0          0          0       0        0        0   
...        ...       ...        ...        ...     ...      ...      ...   
1019         0         0          0          0       0        0        0   
1020         0         0          0          0       0        0        0   
1021         0         0          0          0       0        0        0   
1022         0         0          0          0       0        0        0   
1023         0         0          0          0       0        0        0   

      iap_revenue_d7  iap_revenue_d14  iap_revenue_d28  ...  \
0                0.0    

       buyer_d1  buyer_d7  buyer_d14  buyer_d28  buy_d7  buy_d14  buy_d28  \
29696         0         0          0          0       0        0        0   
29697         0         0          0          0       0        0        0   
29698         0         0          0          0       0        0        0   
29699         0         0          0          0       0        0        0   
29700         0         0          0          0       0        0        0   
...         ...       ...        ...        ...     ...      ...      ...   
30715         0         0          0          0       0        0        0   
30716         0         0          0          0       0        0        0   
30717         0         0          0          0       0        0        0   
30718         0         0          0          0       0        0        0   
30719         0         0          0          0       0        0        0   

       iap_revenue_d7  iap_revenue_d14  iap_revenue_d28  ...  \
29696      

       buyer_d1  buyer_d7  buyer_d14  buyer_d28  buy_d7  buy_d14  buy_d28  \
57344         0         0          0          0       0        0        0   
57345         0         0          0          0       0        0        0   
57346         0         0          0          0       0        0        0   
57347         0         0          0          0       0        0        0   
57348         0         0          0          0       0        0        0   
...         ...       ...        ...        ...     ...      ...      ...   
58363         0         0          0          0       0        0        0   
58364         0         0          0          0       0        0        0   
58365         0         0          0          0       0        0        0   
58366         0         0          0          0       0        0        0   
58367         0         0          0          0       0        0        0   

       iap_revenue_d7  iap_revenue_d14  iap_revenue_d28  ...  \
57344      

       buyer_d1  buyer_d7  buyer_d14  buyer_d28  buy_d7  buy_d14  buy_d28  \
84992         0         0          0          0       0        0        0   
84993         0         0          0          0       0        0        0   
84994         0         0          0          0       0        0        0   
84995         0         0          0          0       0        0        0   
84996         0         0          0          0       0        0        0   
...         ...       ...        ...        ...     ...      ...      ...   
86011         0         0          0          0       0        0        0   
86012         0         0          0          0       0        0        0   
86013         0         0          0          0       0        0        0   
86014         0         0          0          0       0        0        0   
86015         0         0          0          0       0        0        0   

       iap_revenue_d7  iap_revenue_d14  iap_revenue_d28  ...  \
84992      

        buyer_d1  buyer_d7  buyer_d14  buyer_d28  buy_d7  buy_d14  buy_d28  \
110592         0         0          0          0       0        0        0   
110593         0         0          0          0       0        0        0   
110594         0         0          0          0       0        0        0   
110595         0         0          0          0       0        0        0   
110596         0         0          0          0       0        0        0   
...          ...       ...        ...        ...     ...      ...      ...   
111611         0         0          0          0       0        0        0   
111612         0         0          0          0       0        0        0   
111613         0         0          0          0       0        0        0   
111614         0         0          0          0       0        0        0   
111615         0         0          0          0       0        0        0   

        iap_revenue_d7  iap_revenue_d14  iap_revenue_d28  ...  

        buyer_d1  buyer_d7  buyer_d14  buyer_d28  buy_d7  buy_d14  buy_d28  \
138240         0         0          0          0       0        0        0   
138241         0         0          0          0       0        0        0   
138242         0         0          0          0       0        0        0   
138243         0         1          1          1       1        1        1   
138244         0         0          0          0       0        0        0   
...          ...       ...        ...        ...     ...      ...      ...   
139259         0         0          0          0       0        0        0   
139260         0         0          0          0       0        0        0   
139261         0         0          0          0       0        0        0   
139262         0         0          0          0       0        0        0   
139263         0         0          0          0       0        0        0   

        iap_revenue_d7  iap_revenue_d14  iap_revenue_d28  ...  

## Exploratory data analysis

Let's do a exploratory data analysis over the first partition of the Dask DataFrame.

In [15]:
part_0.describe()

,buyer_d1,buyer_d7,buyer_d14,buyer_d28,buy_d7,buy_d14,buy_d28,iap_revenue_d7,iap_revenue_d14,iap_revenue_d28,...,release_msrp,weekday,avg_act_days,avg_days_ins,first_request_ts,last_buy,last_ins,weekend_ratio,weeks_since_first_seen,wifi_ratio
count,121887.000000,121887.000000,121887.000000,121887.000000,121887.000000,121887.000000,121887.000000,121887.000000,121887.000000,121887.000000,...,109905.000000,121887.0,61107.000000,7183.000000,5.390700e+04,2.374000e+03,2.052200e+04,63305.000000,67489.000000,63304.000000
mean,0.031217,0.042301,0.044615,0.046601,0.095096,0.124853,0.174161,8.001834,8.308581,9.071367,...,564.108494,3.0,3.998519,5.228739,1.758773e+09,1.758279e+09,1.758424e+09,0.329158,4.877195,0.641166
std,0.173906,0.201277,0.206458,0.210783,0.996749,1.397024,2.388029,2048.960358,2049.186514,2050.423635,...,514.951762,0.0,2.115821,5.584633,3.757641e+05,7.122233e+05,7.406386e+05,0.270035,1.947677,0.398610
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,30.000000,3.0,1.000000,0.000000,1.758067e+09,1.756600e+09,1.756771e+09,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,180.000000,3.0,2.000000,1.000000,1.758499e+09,1.757735e+09,1.757857e+09,0.129870,4.000000,0.221200
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,299.000000,3.0,4.000000,3.333333,1.758499e+09,1.758478e+09,1.758611e+09,0.279810,6.000000,0.842471
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,899.000000,3.0,6.000000,7.000000,1.759190e+09,1.758888e+09,1.759091e+09,0.458333,6.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,136.000000,150.000000,368.000000,696622.140000,696622.140000,696622.140000,...,5160.000000,3.0,7.000000,28.000000,1.759190e+09,1.759104e+09,1.759277e+09,1.000000,6.000000,1.000000


There are 85 columns. Some are labels, some are features. I am going to separate them.

In [16]:
len(part_0.columns)

85

In [17]:
part_0.columns

Index(['buyer_d1', 'buyer_d7', 'buyer_d14', 'buyer_d28', 'buy_d7', 'buy_d14',
       'buy_d28', 'iap_revenue_d7', 'iap_revenue_d14', 'iap_revenue_d28',
       'registration', 'retention_d1_to_d7', 'retention_d3_to_d7',
       'retention_d7_to_d14', 'retention_d1', 'retention_d3', 'retentiond7',
       'advertiser_bundle', 'advertiser_category', 'advertiser_subcategory',
       'advertiser_bottom_taxonomy_level', 'carrier', 'country', 'region',
       'dev_make', 'dev_model', 'dev_os', 'dev_osv', 'hour', 'release_date',
       'release_msrp', 'weekday', 'avg_act_days', 'avg_daily_sessions',
       'avg_days_ins', 'avg_duration', 'bcat', 'bcat_bottom_taxonomy',
       'bundles_cat', 'bundles_cat_bottom_taxonomy', 'bundles_ins',
       'city_hist', 'country_hist', 'cpm', 'cpm_pct_rk', 'ctr', 'ctr_pct_rk',
       'dev_language_hist', 'dev_osv_hist', 'first_request_ts',
       'first_request_ts_bundle', 'first_request_ts_category_bottom_taxonomy',
       'hour_ratio', 'iap_revenue_usd_bundl

In [18]:
label_names = ['buyer_d1', 'buyer_d7', 'buyer_d14', 'buyer_d28', 'buy_d7', 'buy_d14',
               'buy_d28', 'iap_revenue_d7', 'iap_revenue_d14', 'iap_revenue_d28',
               'registration', 'retention_d1_to_d7', 'retention_d3_to_d7',
               'retention_d7_to_d14', 'retention_d1', 'retention_d3', 'retentiond7']

request_features = ['advertiser_bundle', 'advertiser_category', 'advertiser_subcategory',
                    'advertiser_bottom_taxonomy_level', 'carrier', 'country', 'region',
                    'dev_make', 'dev_model', 'dev_os', 'dev_osv', 'hour', 'release_date',
                    'release_msrp', 'weekday']

user_features =   ['avg_act_days', 'avg_daily_sessions',
                   'avg_days_ins', 'avg_duration', 'bcat', 'bcat_bottom_taxonomy',
                   'bundles_cat', 'bundles_cat_bottom_taxonomy', 'bundles_ins',
                   'city_hist', 'country_hist', 'cpm', 'cpm_pct_rk', 'ctr', 'ctr_pct_rk',
                   'dev_language_hist', 'dev_osv_hist', 'first_request_ts',
                   'first_request_ts_bundle', 'first_request_ts_category_bottom_taxonomy',
                   'hour_ratio', 'iap_revenue_usd_bundle', 'iap_revenue_usd_category',
                   'iap_revenue_usd_category_bottom_taxonomy', 'last_buy',
                   'last_buy_ts_bundle', 'last_buy_ts_category', 'last_ins',
                   'last_install_ts_bundle', 'last_install_ts_category',
                   'advertiser_actions_action_count',
                   'advertiser_actions_action_last_timestamp',
                   'user_actions_bundles_action_count',
                   'user_actions_bundles_action_last_timestamp', 'last_advertiser_action',
                   'new_bundles', 'num_buys_bundle', 'num_buys_category',
                   'num_buys_category_bottom_taxonomy', 'region_hist', 'rev_by_adv',
                   'rwd_prank', 'user_bundles', 'user_bundles_l28d', 'weekend_ratio',
                   'weeks_since_first_seen', 'wifi_ratio',
                   'whale_users_bundle_num_buys_prank', 'whale_users_bundle_revenue_prank',
                   'whale_users_bundle_total_num_buys', 'whale_users_bundle_total_revenue']

aux_names = ['row_id', 'datetime']

### User related features

A lot of these features are of variable complexity, as some of them correspond to lists of values depending on the bundles used by each user. We will need to apply one-hot encoding to these features.

Furthermore, there are a lot of missing values. We define the NA replacement policy here:

### NaNs

#### Removal

We remove rows that are just all NaNs.

#### Replace by mean

'avg_act_days', 'weekend_ratio', 'wiki_ratio'

#### Replace by max

'avg_days_ins', 'weeks_since_first_seen'

### Feature engineering

#### Separation

'bcat', 'bcat_bottom_taxonomy', 'bundles_cat', 'bundles_cat_bottom_taxonomy', 'country', 'cpm', 'cpm_pct_rk', 'ctr', 'ctr_pct_rk', 'dev_language_hist', 'iap_revenue_usd_category', 'iap_revenue_usd_category_bottom_taxonomy', 'num_buys_category', 'num_buys_category_bottom_taxonomy'

#### One-hot encoding

'last_advertiser_action'

#### Replacement of features

By sum:
'avg_daily_sessions', 'advertiser_actions_action_count', 'user_actions_bundles_action_count', 'whale_users_bundle_total_num_buys', 'whale_users_bundle_total_revenue'

By mean:
'avg_duration', 'rev_by_adv', 'rwd_prank', 'whale_users_bundle_num_buys_prank', 'whale_users_bundle_revenue_prank'

#### Removal of features

'bundles_ins', 'city_hist', 'region_hist', 'dev_osv_hist', 'first_request_ts', 'first_request_ts_bundle', 'first_request_ts_category_bottom_taxonomy', 'iap_revenue_usd_bundle', 'last_buy', 'last_buy_ts_bundle', 'last_buy_ts_category', 'hour_ratio', 'last_ins', 'last_install_ts_bundle', 'last_install_ts_category', 'advertiser_actions_action_last_timestamp', 'user_actions_bundles_action_last_timestamp', 'new_bundles', 'num_buys_bundle', 'user_bundles', 'user_bundles_l28d'


In [19]:
jason_good_names = []

jason_bad_names = []

In [20]:
user_df = part_0.loc[:, user_features]

In [21]:
user_df['avg_act_days'].describe()

count    61107.000000
mean         3.998519
std          2.115821
min          1.000000
25%          2.000000
50%          4.000000
75%          6.000000
max          7.000000
Name: avg_act_days, dtype: float64

In [22]:
user_df.loc[user_df['avg_daily_sessions'].notna(), "avg_daily_sessions"]

2         [(0e0c94b1bab6c95fe79511525d24aefccb754f08, 1)...
4         [(2706122d1fba30ded1e9fa80951afc6e0c711017, 5)...
6           [(d8538ce27256d97f754711d9c9dd4ea533a113c8, 1)]
7         [(3b91538ba98ced2700b3a53cb84df92d072e8df8, 1)...
10          [(38d84fdd4ba87095fa23a4f4138352d4ed414ef8, 1)]
                                ...                        
121870      [(2b6c0bb1833d2bb8a7f313f1263d36eac7b300b2, 1)]
121873      [(7722e46ae98798bc2c0ad2f247ab5aa2684abd11, 1)]
121878    [(8b77d59d78da2fcaf14230360c0fedec453c4fc2, 2)...
121879      [(15452781dc92b27820ded432a3fcd47992b3a39d, 2)]
121885      [(3cab54b2b66a809e4ace91337d64ef0098949f47, 1)]
Name: avg_daily_sessions, Length: 29142, dtype: object

In [23]:
user_df['avg_days_ins'].describe()

count    7183.000000
mean        5.228739
std         5.584633
min         0.000000
25%         1.000000
50%         3.333333
75%         7.000000
max        28.000000
Name: avg_days_ins, dtype: float64

In [24]:
user_df.loc[user_df['iap_revenue_usd_category_bottom_taxonomy'].notna(), "iap_revenue_usd_category_bottom_taxonomy"]

14        [(Family & Parenting Shopping (General), 401.5...
41               [(Shopping (General), 244.32999999999998)]
44          [(Slots, 332.2159534400001), (Solitaire, 12.0)]
48                   [(Hidden Objects, 1.99), (Slots, 0.0)]
58                                                       []
                                ...                        
121745                    [(Carpooling & Ridesharing, 0.0)]
121751                          [(Shopping (General), 0.0)]
121814              [(Restaurant Ordering & Delivery, 0.0)]
121820                [(Tower Defense, 26.940000000000005)]
121878                                                   []
Name: iap_revenue_usd_category_bottom_taxonomy, Length: 5603, dtype: object

In [25]:
user_df['avg_daily_sessions'].describe()

count                                               29142
unique                                              20084
top       [(93d3de5c6eb6525790ac30975a3c36597534d5dd, 1)]
freq                                                  555
Name: avg_daily_sessions, dtype: object

In [26]:
user_df.loc[2, 'avg_daily_sessions']

[('0e0c94b1bab6c95fe79511525d24aefccb754f08', 1),
 ('8da885dfc28811f5e9f8418511ca6fc989d85913', 1),
 ('221d7c94e54530510ec40b174a44cc60318dff55', 1),
 ('e1260e164a10de7d01ff1cd7ecb702d1f0fd3853', 1),
 ('150045628befad0948138e18daff2905ab44988e', 1)]

In [27]:
user_df.loc[0, 'bundles_ins']

array(['d54f414516df4ba311d4b2a67b2d895175ea285b',
       '0de540d0711175dd3a4ffa7e7f8450abc7870328'], dtype=object)

In [28]:
user_df['cpm']

0         [(r, 324.06158940143393), (i, 268.113535940146...
1                                                      None
2         [(r, 49.498038189707096), (b, 3.06099328974128...
3                                                      None
4         [(b, 0.9611235379166666), (i, 5.25769010296437...
                                ...                        
121882                                                 None
121883                                      [(i, 0.140168)]
121884                                                 None
121885                                                 None
121886            [(b, 2.2859675), (i, 119.85426846579999)]
Name: cpm, Length: 121887, dtype: object

In [29]:
user_df['cpm_pct_rk']

0         [(i, 0.8640278470866332), (r, 0.82157364224462...
1                                                      None
2         [(r, 0.5823951655039672), (b, 0.56201751419578...
3                                                      None
4         [(r, 0.09386688629673481), (b, 0.2888176593734...
                                ...                        
121882                                                 None
121883                           [(i, 0.27616875712656785)]
121884                                                 None
121885                                                 None
121886    [(i, 0.9158277489044964), (b, 0.60324480927088...
Name: cpm_pct_rk, Length: 121887, dtype: object

In [30]:
user_df['ctr']

0               None
1               None
2               None
3               None
4         [(i, 1.0)]
             ...    
121882          None
121883          None
121884          None
121885          None
121886          None
Name: ctr, Length: 121887, dtype: object

In [31]:
user_df['ctr_pct_rk']

0                              None
1                              None
2                              None
3                              None
4         [(i, 0.6347302774841429)]
                    ...            
121882                         None
121883                         None
121884                         None
121885                         None
121886                         None
Name: ctr_pct_rk, Length: 121887, dtype: object

In [32]:
user_df['dev_language_hist']

0                    [(en, 534)]
1                           None
2                   [(en, 4173)]
3                    [(ar, 371)]
4         [(en, 17653), (es, 1)]
                   ...          
121882                      None
121883                [(en, 11)]
121884                      None
121885                [(ko, 21)]
121886              [(en, 1550)]
Name: dev_language_hist, Length: 121887, dtype: object

In [33]:
user_df['dev_osv_hist']

0                                           [(15.0.0, 534)]
1                                                      None
2                                          [(18.4.1, 4173)]
3                                           [(14.0.0, 371)]
4         [(15.0.0, 17640), (8.1.0, 1), (13.0.0, 2), (11...
                                ...                        
121882                                                 None
121883                           [(14.0.0, 4), (15.0.0, 7)]
121884                                                 None
121885                          [(18.6.0, 15), (26.0.0, 6)]
121886                      [(14.0.0, 1114), (15.0.0, 436)]
Name: dev_osv_hist, Length: 121887, dtype: object

In [34]:
user_df['first_request_ts']

0         1.759190e+09
1                  NaN
2         1.759190e+09
3         1.758240e+09
4         1.759190e+09
              ...     
121882             NaN
121883    1.758499e+09
121884             NaN
121885    1.758499e+09
121886             NaN
Name: first_request_ts, Length: 121887, dtype: float64

In [35]:
user_df['first_request_ts_bundle']

0         [(d026805c95a7fa27fcae2a72fd5d6d092b7653bf, 17...
1                                                      None
2         [(8da885dfc28811f5e9f8418511ca6fc989d85913, 17...
3         [(572f4ea557dbb90dad815064a616d7d7f76a939c, 17...
4         [(7c17d599a2b56ce0252d8a062ca6d33bc99ef5d2, 17...
                                ...                        
121882                                                 None
121883    [(54662cebb1f386a180887921e549c9622d6896a6, 17...
121884                                                 None
121885    [(e001e73e66372eff1144511c7d5568100bb6c36c, 17...
121886                                                 None
Name: first_request_ts_bundle, Length: 121887, dtype: object

In [36]:
user_df.loc[user_df['hour_ratio'].notna(), "hour_ratio"]

0         [(5, 0.3118811881188119), (14, 0.0915841584158...
2         [(10, 0.04834761321909425), (14, 0.00061199510...
3         [(0, 0.13259668508287292), (10, 0.060773480662...
4         [(0, 7.181328545780969e-05), (10, 0.0385637342...
6         [(10, 0.07142857142857142), (14, 0.01785714285...
                                ...                        
121879    [(0, 0.03503184713375796), (5, 0.0254777070063...
121880    [(10, 0.03896103896103896), (14, 0.07792207792...
121883    [(17, 0.14285714285714285), (19, 0.14285714285...
121885    [(14, 0.3333333333333333), (20, 0.142857142857...
121886    [(0, 0.03779527559055118), (5, 0.0440944881889...
Name: hour_ratio, Length: 63305, dtype: object

In [37]:
user_df['iap_revenue_usd_bundle']

0         None
1         None
2         None
3         None
4         None
          ... 
121882    None
121883    None
121884    None
121885    None
121886    None
Name: iap_revenue_usd_bundle, Length: 121887, dtype: object

In [38]:
user_df.loc[user_df['iap_revenue_usd_bundle'].notna(), "iap_revenue_usd_bundle"]

14        [(163ffe25c9eb6e1d5702e6ae5e539f9b570bbdf1, 40...
41        [(0275805fca51acc78aa5a0428b34d4132e63bebd, 24...
44        [(3ded77f26545802d8fe2c7ffcfc5c1c967c0de1f, 17...
48        [(3dc8db5d6618be2b7dd0eccca6b3e19b83311377, 1....
58        [(b6362eb69435662dd1245a80900d44b631099144, 4....
                                ...                        
121745    [(0ea839df44a85045f50c64e48dfff3dcae72175a, 0.0)]
121751    [(04de97466acac04d45806b61ce71590b128d550d, 0.0)]
121814    [(08b8adea9ff61f599c37db51e73ddf3ae12ea497, 0.0)]
121820    [(21afeae49ba828fb27842e7eac98302e04bcd37b, 26...
121878    [(ca673c2de075f7d82723f0831e641032241b1b86, 14...
Name: iap_revenue_usd_bundle, Length: 5603, dtype: object

In [39]:
user_df.loc[user_df['iap_revenue_usd_category'].notna(), "iap_revenue_usd_category"]

14                        [(Shopping-, 401.55124261939136)]
41                [(Shopping-Business, 244.32999999999998)]
44        [(Games-, 332.2159534400001), (Games-Games/Cas...
48        [(Games-Games/Adventure, 1.99), (Games-Games/C...
58                                                       []
                                ...                        
121745                  [(Application-Travel & Local, 0.0)]
121751                        [(Application-Shopping, 0.0)]
121814                    [(Application-Food & Drink, 0.0)]
121820                [(Game-Strategy, 26.940000000000005)]
121878                     [(Application-Lifestyle, 14.25)]
Name: iap_revenue_usd_category, Length: 5603, dtype: object

In [40]:
user_df.loc[user_df['iap_revenue_usd_category_bottom_taxonomy'].notna(), "iap_revenue_usd_category_bottom_taxonomy"]

14        [(Family & Parenting Shopping (General), 401.5...
41               [(Shopping (General), 244.32999999999998)]
44          [(Slots, 332.2159534400001), (Solitaire, 12.0)]
48                   [(Hidden Objects, 1.99), (Slots, 0.0)]
58                                                       []
                                ...                        
121745                    [(Carpooling & Ridesharing, 0.0)]
121751                          [(Shopping (General), 0.0)]
121814              [(Restaurant Ordering & Delivery, 0.0)]
121820                [(Tower Defense, 26.940000000000005)]
121878                                                   []
Name: iap_revenue_usd_category_bottom_taxonomy, Length: 5603, dtype: object

In [41]:
user_df.loc[user_df["last_advertiser_action"].notna(), "last_advertiser_action"]

209               af_ad_view
264            af_app_opened
294            af_app_opened
431       pltv_abyssal_1_iap
467            af_app_opened
                 ...        
121547         af_app_opened
121600         af_app_opened
121610         af_app_opened
121625         af_app_opened
121831         af_app_opened
Name: last_advertiser_action, Length: 1341, dtype: object

In [42]:
user_df.loc[user_df["num_buys_category"].notna(), "num_buys_category"]

14                                        [(Shopping-, 23)]
41                                 [(Shopping-Business, 5)]
44        [(Games-, 48), (Games-Games/Casino, 2), (Games...
48        [(Games-Games/Casino, 1), (Games-Games/Adventu...
58                                                       []
                                ...                        
121745                    [(Application-Travel & Local, 7)]
121751                         [(Application-Shopping, 42)]
121814                      [(Application-Food & Drink, 1)]
121820                                 [(Game-Strategy, 6)]
121878                         [(Application-Lifestyle, 6)]
Name: num_buys_category, Length: 5603, dtype: object

In [43]:
user_df["new_bundles"]

0         [d026805c95a7fa27fcae2a72fd5d6d092b7653bf, 43e...
1                                                      None
2         [8da885dfc28811f5e9f8418511ca6fc989d85913, 221...
3                [572f4ea557dbb90dad815064a616d7d7f76a939c]
4         [7c17d599a2b56ce0252d8a062ca6d33bc99ef5d2, 55e...
                                ...                        
121882                                                 None
121883           [54662cebb1f386a180887921e549c9622d6896a6]
121884                                                 None
121885    [e001e73e66372eff1144511c7d5568100bb6c36c, 6ad...
121886                                                 None
Name: new_bundles, Length: 121887, dtype: object

In [44]:
user_df["region_hist"]

0                                [(US-CA, 529), (US-PA, 5)]
1                                                      None
2                [(US-IN, 116), (US-CA, 15), (US-TX, 4042)]
3                                            [(YE-AD, 371)]
4         [(US-NY, 7294), (US-UT, 2), (US-MD, 4), (US-FL...
                                ...                        
121882                                                 None
121883                                        [(PK-PB, 11)]
121884                                                 None
121885                                        [(KR-11, 21)]
121886                          [(US-NV, 2), (US-CA, 1548)]
Name: region_hist, Length: 121887, dtype: object

In [45]:
user_df.loc[user_df["rev_by_adv"].notna(), "rev_by_adv"]

23        [(1bb185abf22f02aafcebb9727eab3e6bf4120523, 0....
33        [(3221d5bbb1a2f36b5a0fe24be198728837774c55, 0....
55        [(82162cba5d9a28310992951a6342a1acd1c385e8, 0....
162       [(75ff18049036c5dc13a96771b2a075d25f5a1959, 0....
246       [(001b729f996da29d01e9e3641ac7fb153e6bc98e, 0....
                                ...                        
121382    [(001b729f996da29d01e9e3641ac7fb153e6bc98e, 0....
121466    [(001b729f996da29d01e9e3641ac7fb153e6bc98e, 0....
121539    [(371932ceab23d320aba11487c09185578d38d767, 0.0)]
121553    [(76538f92b80a4519a392d3176e5cfe6148b51c4b, 0....
121687    [(001b729f996da29d01e9e3641ac7fb153e6bc98e, 0....
Name: rev_by_adv, Length: 2106, dtype: object

In [46]:
user_df['weekend_ratio']

0         0.019802
1              NaN
2         0.399021
3         0.121547
4         0.293285
            ...   
121882         NaN
121883    0.000000
121884         NaN
121885    0.142857
121886    0.332903
Name: weekend_ratio, Length: 121887, dtype: float64

In [47]:
user_df['weeks_since_first_seen']

0         6.0
1         NaN
2         6.0
3         6.0
4         6.0
         ... 
121882    NaN
121883    6.0
121884    NaN
121885    2.0
121886    3.0
Name: weeks_since_first_seen, Length: 121887, dtype: float64

In [48]:
user_df['weeks_since_first_seen'].describe()

count    67489.000000
mean         4.877195
std          1.947677
min          0.000000
25%          4.000000
50%          6.000000
75%          6.000000
max          6.000000
Name: weeks_since_first_seen, dtype: float64

In [49]:
user_df['wifi_ratio']

0         0.913366
1              NaN
2         0.999388
3         1.000000
4         0.160243
            ...   
121882         NaN
121883    0.285714
121884         NaN
121885    0.095238
121886    0.005161
Name: wifi_ratio, Length: 121887, dtype: float64

In [50]:
user_df['wifi_ratio'].describe()

count    63304.000000
mean         0.641166
std          0.398610
min          0.000000
25%          0.221200
50%          0.842471
75%          1.000000
max          1.000000
Name: wifi_ratio, dtype: float64

In [51]:
user_df.loc[user_df['whale_users_bundle_num_buys_prank'].notna(), 'whale_users_bundle_num_buys_prank']

14        [(163ffe25c9eb6e1d5702e6ae5e539f9b570bbdf1, 0....
41        [(0275805fca51acc78aa5a0428b34d4132e63bebd, 0....
44        [(3ded77f26545802d8fe2c7ffcfc5c1c967c0de1f, 0....
48        [(3bbb191b7dd04723e564c014d8d8595056e7fe1c, 0....
58        [(b6362eb69435662dd1245a80900d44b631099144, 0.0)]
                                ...                        
121680    [(73ea4d46b41ece6af5e4a5b8a8a76e0b7474342b, 0....
121683    [(e73cc8ad7df8eacb6141f06f7ce9b53ea1856387, 0.0)]
121720    [(3714e810963165f514bbeec2cc0b54e91239314b, 0.0)]
121820    [(21afeae49ba828fb27842e7eac98302e04bcd37b, 0....
121878    [(ca673c2de075f7d82723f0831e641032241b1b86, 0....
Name: whale_users_bundle_num_buys_prank, Length: 4488, dtype: object

In [52]:
user_df.loc[user_df['whale_users_bundle_revenue_prank'].notna(), 'whale_users_bundle_revenue_prank']

14        [(163ffe25c9eb6e1d5702e6ae5e539f9b570bbdf1, 0....
41        [(0275805fca51acc78aa5a0428b34d4132e63bebd, 0....
44        [(a17c7b7aa69f94086f5dfb21cf136ae67b7b95d6, 0....
48        [(3bbb191b7dd04723e564c014d8d8595056e7fe1c, 0....
58        [(b6362eb69435662dd1245a80900d44b631099144, 0....
                                ...                        
121680    [(73ea4d46b41ece6af5e4a5b8a8a76e0b7474342b, 0....
121683    [(e73cc8ad7df8eacb6141f06f7ce9b53ea1856387, 0....
121720    [(3714e810963165f514bbeec2cc0b54e91239314b, 0....
121820    [(21afeae49ba828fb27842e7eac98302e04bcd37b, 0....
121878    [(ca673c2de075f7d82723f0831e641032241b1b86, 0....
Name: whale_users_bundle_revenue_prank, Length: 4488, dtype: object

In [53]:
user_df.loc[user_df['whale_users_bundle_total_num_buys'].notna(), 'whale_users_bundle_total_num_buys']

14         [(163ffe25c9eb6e1d5702e6ae5e539f9b570bbdf1, 23)]
41          [(0275805fca51acc78aa5a0428b34d4132e63bebd, 5)]
44        [(3ded77f26545802d8fe2c7ffcfc5c1c967c0de1f, 28...
48        [(3bbb191b7dd04723e564c014d8d8595056e7fe1c, 19...
58          [(b6362eb69435662dd1245a80900d44b631099144, 1)]
                                ...                        
121680      [(73ea4d46b41ece6af5e4a5b8a8a76e0b7474342b, 8)]
121683      [(e73cc8ad7df8eacb6141f06f7ce9b53ea1856387, 1)]
121720      [(3714e810963165f514bbeec2cc0b54e91239314b, 1)]
121820      [(21afeae49ba828fb27842e7eac98302e04bcd37b, 6)]
121878      [(ca673c2de075f7d82723f0831e641032241b1b86, 6)]
Name: whale_users_bundle_total_num_buys, Length: 4488, dtype: object

In [54]:
user_df.loc[user_df['whale_users_bundle_total_revenue'].notna(), 'whale_users_bundle_total_revenue']

14        [(163ffe25c9eb6e1d5702e6ae5e539f9b570bbdf1, 40...
41        [(0275805fca51acc78aa5a0428b34d4132e63bebd, 24...
44        [(3ded77f26545802d8fe2c7ffcfc5c1c967c0de1f, 17...
48        [(3bbb191b7dd04723e564c014d8d8595056e7fe1c, 50...
58        [(b6362eb69435662dd1245a80900d44b631099144, 4....
                                ...                        
121680    [(73ea4d46b41ece6af5e4a5b8a8a76e0b7474342b, 65...
121683    [(e73cc8ad7df8eacb6141f06f7ce9b53ea1856387, 1....
121720    [(3714e810963165f514bbeec2cc0b54e91239314b, 1....
121820    [(21afeae49ba828fb27842e7eac98302e04bcd37b, 26...
121878    [(ca673c2de075f7d82723f0831e641032241b1b86, 14...
Name: whale_users_bundle_total_revenue, Length: 4488, dtype: object

Bundles ins, avg_daily_sessions, avg_duration

??:
City_hist
Country_hist more interesting than previous

In [55]:
user_df.describe()

,avg_act_days,avg_days_ins,first_request_ts,last_buy,last_ins,weekend_ratio,weeks_since_first_seen,wifi_ratio
count,61107.000000,7183.000000,5.390700e+04,2.374000e+03,2.052200e+04,63305.000000,67489.000000,63304.000000
mean,3.998519,5.228739,1.758773e+09,1.758279e+09,1.758424e+09,0.329158,4.877195,0.641166
std,2.115821,5.584633,3.757641e+05,7.122233e+05,7.406386e+05,0.270035,1.947677,0.398610
min,1.000000,0.000000,1.758067e+09,1.756600e+09,1.756771e+09,0.000000,0.000000,0.000000
25%,2.000000,1.000000,1.758499e+09,1.757735e+09,1.757857e+09,0.129870,4.000000,0.221200
50%,4.000000,3.333333,1.758499e+09,1.758478e+09,1.758611e+09,0.279810,6.000000,0.842471
75%,6.000000,7.000000,1.759190e+09,1.758888e+09,1.759091e+09,0.458333,6.000000,1.000000
max,7.000000,28.000000,1.759190e+09,1.759104e+09,1.759277e+09,1.000000,6.000000,1.000000


### Request related features

### NaNs

#### Removal

#### Replace by mean

#### Replace by max

### Feature engineering

#### One-hot encoding

'advertiser_category', 'advertiser_subcategory', 'advertiser_bottom_taxonomy_level', 'country', 'dev_make', 'dev_os', 'release_msrp'

#### Removal of features

'advertiser_bundle', 'carrier', 'region', 'dev_model', 'dev_osv', 'hour', 'release_date', 'release_msrp', 'weekday'

In [57]:
request_features = ['advertiser_bundle', 'advertiser_category', 'advertiser_subcategory',
                    'advertiser_bottom_taxonomy_level', 'carrier', 'country', 'region',
                    'dev_make', 'dev_model', 'dev_os', 'dev_osv', 'hour', 'release_date',
                    'release_msrp', 'weekday']

part_0['advertiser_bundle']

0         26fc75598f3a2e63782b4166883034072c49b4aa
1         7eaeef21e215654ea75a5ef78692d1f8ffd0ca54
2         7c60a4ded8a554bf3cfba714098f6237364556ff
3         466d0d44d8e69adb8e4425de93d23317bd5304f6
4         c811547cfd9a6efc65c450c11fdd7db8ad9f9871
                            ...                   
121882    7eaeef21e215654ea75a5ef78692d1f8ffd0ca54
121883    001b729f996da29d01e9e3641ac7fb153e6bc98e
121884    db42749ef73fd488a95e37718f033eab132da07c
121885    d238ae3ee12dcef245f58f4813b79021c0a9155e
121886    fcce26ae1abe348cb24c4f4e542957ef5d9773fd
Name: advertiser_bundle, Length: 121887, dtype: object

In [60]:
part_0['advertiser_category']

count            110633
unique               21
top       sport betting
freq              28472
Name: advertiser_category, dtype: object

In [63]:
part_0['advertiser_category'].unique()

array(['game', 'sport betting', 'games', None, 'application',
       'food & drink', 'finance', 'social networking', 'lifestyle',
       'real money casino', 'photo & video', 'shopping',
       'health & fitness', 'travel', 'productivity', 'entertainment',
       'utilities', 'education', 'sports', 'lottery', 'books',
       'navigation'], dtype=object)

In [61]:
part_0['advertiser_subcategory']
#'advertiser_category', 'advertiser_subcategory', 'advertiser_bottom_taxonomy_level', 'country', 'dev_make', 'dev_os', 'release_msrp'

0               skill game
1                  unknown
2          games/adventure
3                  unknown
4                   puzzle
                ...       
121882             unknown
121883              casual
121884       communication
121885    games/simulation
121886       entertainment
Name: advertiser_subcategory, Length: 121887, dtype: object

In [62]:
part_0['advertiser_subcategory'].unique()

array(['skill game', 'unknown', 'games/adventure', 'puzzle', None,
       'games/puzzle', 'skill games', 'finance', 'tools', 'casino',
       'games/action', 'games/hypercasual', 'communication', 'casual',
       'travel & local', 'word', 'games/board', 'games/racing',
       'utilities', 'games/casual match 3', 'entertainment',
       'games/sports', 'play2earn', 'dating', 'shopping', 'simulation',
       'lifestyle', 'games/casual', 'strategy', 'arcade', 'role playing',
       'games/casino', 'action', 'games/simulation', 'food & drink',
       'casual match 3', 'social', 'travel', 'sports',
       'games/role playing', 'card', 'adventure', 'racing', 'education',
       'navigation', 'health & fitness', 'games/strategy',
       'social networking', 'games/card', 'productivity', 'board',
       'books', 'business', 'games/family', 'games/word', 'games',
       'newsstand'], dtype=object)

In [65]:
part_0['advertiser_bottom_taxonomy_level'].describe()

count          73663
unique            82
top       match swap
freq           18322
Name: advertiser_bottom_taxonomy_level, dtype: object

In [64]:
part_0['advertiser_bottom_taxonomy_level'].unique()

array([None, 'geolocation', 'sports entertainment (general)',
       'match swap', 'match pair', 'slots',
       'investing & financial management (general)', 'antivirus & vpn',
       'fps / 3ps', 'real-time strategy', 'social messaging',
       'accommodations & lodging (general)', 'word',
       'restaurant ordering & delivery',
       'fx (foreign exchange) trading & markets', 'coin looters',
       'action sandbox', 'cryptocurrency', 'bingo', 'realistic sports',
       'match merge 3', 'dating', 'coupons & rebates (general)',
       'sports & outdoors (general)', 'breeding',
       'selfie & beauty editors', 'match merge 2', 'idler',
       'tower defense', 'pinball / brick breaker',
       'team sports & outdoor recreation', 'squad rpg', 'other casino',
       'bus & rail services', 'tycoon / crafting', 'solitaire',
       'food delivery services (general)',
       'travel booking services & travel agencies (general)',
       'avatar life', 'empire simulation', 'match blast',
   

In [ ]:
part_0['advertiser_bundle'].describe()

### All features

### NaNs

#### Removal

We remove rows that are just all NaNs.

#### Replace by mean

'avg_act_days', 'weekend_ratio', 'wifi_ratio'

#### Replace by max

'avg_days_ins', 'weeks_since_first_seen'

### Feature engineering

#### Separation

'bcat', 'bcat_bottom_taxonomy', 'bundles_cat', 'bundles_cat_bottom_taxonomy', 'country', 'cpm', 'cpm_pct_rk', 'ctr', 'ctr_pct_rk', 'dev_language_hist', 'iap_revenue_usd_category', 'iap_revenue_usd_category_bottom_taxonomy', 'num_buys_category', 'num_buys_category_bottom_taxonomy'

#### One-hot encoding

'last_advertiser_action', 'advertiser_category', 'advertiser_subcategory', 'advertiser_bottom_taxonomy_level', 'country', 'dev_make', 'dev_os', 'release_msrp'

#### Replacement of features

By sum:
'avg_daily_sessions', 'advertiser_actions_action_count', 'user_actions_bundles_action_count', 'whale_users_bundle_total_num_buys', 'whale_users_bundle_total_revenue'

By mean:
'avg_duration', 'rev_by_adv', 'rwd_prank', 'whale_users_bundle_num_buys_prank', 'whale_users_bundle_revenue_prank'

#### Removal of features

'bundles_ins', 'city_hist', 'region_hist', 'dev_osv_hist', 'first_request_ts', 'first_request_ts_bundle', 'first_request_ts_category_bottom_taxonomy', 'iap_revenue_usd_bundle', 'last_buy', 'last_buy_ts_bundle', 'last_buy_ts_category', 'hour_ratio', 'last_ins', 'last_install_ts_bundle', 'last_install_ts_category', 'advertiser_actions_action_last_timestamp', 'user_actions_bundles_action_last_timestamp', 'new_bundles', 'num_buys_bundle', 'user_bundles', 'user_bundles_l28d', 'advertiser_bundle', 'carrier', 'region', 'dev_model', 'dev_osv', 'hour', 'release_date', 'release_msrp', 'weekday'

## Pre-processing

### Remove columns (unnecessaries)

In [67]:
part_0_v2 = part_0.drop(columns = ['bundles_ins', 'city_hist', 'region_hist', 'dev_osv_hist', 'first_request_ts', 'first_request_ts_bundle', 'first_request_ts_category_bottom_taxonomy', 'iap_revenue_usd_bundle', 'last_buy', 'last_buy_ts_bundle', 'last_buy_ts_category', 'hour_ratio', 'last_ins', 'last_install_ts_bundle', 'last_install_ts_category', 'advertiser_actions_action_last_timestamp', 'user_actions_bundles_action_last_timestamp', 'new_bundles', 'num_buys_bundle', 'user_bundles', 'user_bundles_l28d', 'advertiser_bundle', 'carrier', 'region', 'dev_model', 'dev_osv', 'hour', 'release_date', 'release_msrp', 'weekday'])
part_0_v2.columns
part_0_v2.columns.shape

(55,)

### Removal of fully NaN rows

In [74]:
user_df_v2 = part_0_v2.drop(columns = list(set(part_0_v2.columns) - set(user_features)))

In [75]:
user_df_v2.columns

Index(['avg_act_days', 'avg_daily_sessions', 'avg_days_ins', 'avg_duration',
       'bcat', 'bcat_bottom_taxonomy', 'bundles_cat',
       'bundles_cat_bottom_taxonomy', 'country_hist', 'cpm', 'cpm_pct_rk',
       'ctr', 'ctr_pct_rk', 'dev_language_hist', 'iap_revenue_usd_category',
       'iap_revenue_usd_category_bottom_taxonomy',
       'advertiser_actions_action_count', 'user_actions_bundles_action_count',
       'last_advertiser_action', 'num_buys_category',
       'num_buys_category_bottom_taxonomy', 'rev_by_adv', 'rwd_prank',
       'weekend_ratio', 'weeks_since_first_seen', 'wifi_ratio',
       'whale_users_bundle_num_buys_prank', 'whale_users_bundle_revenue_prank',
       'whale_users_bundle_total_num_buys',
       'whale_users_bundle_total_revenue'],
      dtype='object')

In [77]:
user_df_v2

,avg_act_days,avg_daily_sessions,avg_days_ins,avg_duration,bcat,bcat_bottom_taxonomy,bundles_cat,bundles_cat_bottom_taxonomy,country_hist,cpm,...,num_buys_category_bottom_taxonomy,rev_by_adv,rwd_prank,weekend_ratio,weeks_since_first_seen,wifi_ratio,whale_users_bundle_num_buys_prank,whale_users_bundle_revenue_prank,whale_users_bundle_total_num_buys,whale_users_bundle_total_revenue
0,2.0,None,17.0,None,"[(Game/Card, 241)]","[(Solitaire, 9)]","[(Game/Card, 2)]","[(Solitaire, 1)]","[(US, 534)]","[(r, 324.06158940143393), (i, 268.113535940146...",...,None,None,"[(25cdfd75fffab2e1953195cca3260a6f9d2b5d5d, 0....",0.019802,6.0,0.913366,None,None,None,None
1,NaN,None,NaN,None,None,None,None,None,None,None,...,None,None,None,NaN,NaN,NaN,None,None,None,None
2,4.0,"[(0e0c94b1bab6c95fe79511525d24aefccb754f08, 1)...",NaN,"[(0e0c94b1bab6c95fe79511525d24aefccb754f08, 0)...","[(Games/Games/Simulation, 5), (Entertainment/U...","[(Driving / Flight Simulator, 5), (Riddle, 146...","[(Entertainment, 1), (Games/Games/Adventure, 1...","[(Riddle, 1), (Customization, 1), (Simulator, 2)]","[(US, 4173)]","[(r, 49.498038189707096), (b, 3.06099328974128...",...,None,None,"[(221d7c94e54530510ec40b174a44cc60318dff55, 0....",0.399021,6.0,0.999388,None,None,None,None
3,3.0,None,NaN,None,"[(Application/Tools, 19)]","[(Cloud Storage, 19)]","[(Application/Tools, 1)]","[(Cloud Storage, 1)]","[(YE, 371)]",None,...,None,None,"[(3ba364431ad52210ba9b6286521264faecc93b34, 0....",0.121547,6.0,1.000000,None,None,None,None
4,7.0,"[(2706122d1fba30ded1e9fa80951afc6e0c711017, 5)...",NaN,"[(2706122d1fba30ded1e9fa80951afc6e0c711017, 18...","[(Game/Puzzle, 5062), (Game/Arcade, 1), (Game/...","[(Word, 4574), (Block, 5058), (Sort, 4), (Arca...","[(Game/Puzzle, 1), (Application/Tools, 1), (Ga...","[(Word, 1), (Block, 1)]","[(US, 17658)]","[(b, 0.9611235379166666), (i, 5.25769010296437...",...,None,None,"[(1031535cf2a1315422fd05d321349bcd3c3ffc04, 0....",0.293285,6.0,0.160243,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121882,NaN,None,NaN,None,None,None,None,None,None,None,...,None,None,None,NaN,NaN,NaN,None,None,None,None
121883,2.0,None,NaN,None,"[(Application/Music & Audio, 3)]","[(Music & Podcasts (General), 3)]",None,None,"[(PK, 11)]","[(i, 0.140168)]",...,None,None,"[(54662cebb1f386a180887921e549c9622d6896a6, 0.0)]",0.000000,6.0,0.285714,None,None,None,None
121884,NaN,None,NaN,None,None,None,None,None,None,None,...,None,None,None,NaN,NaN,NaN,None,None,None,None
121885,2.0,"[(3cab54b2b66a809e4ace91337d64ef0098949f47, 1)]",NaN,"[(3cab54b2b66a809e4ace91337d64ef0098949f47, 887)]","[(Social Networking, 3), (Shopping/Lifestyle, ...","[(Social Media (General), 3), (Buy & Sell (Gen...","[(Navigation, 1), (Photo & Video, 1)]","[(Selfie & Beauty Editors, 1), (Transportation...","[(KR, 21)]",None,...,None,None,"[(3cab54b2b66a809e4ace91337d64ef0098949f47, 0.0)]",0.142857,2.0,0.095238,None,None,None,None


In [86]:
user_df_v3 = user_df_v2.dropna(how = 'all')

In [90]:
user_df_v3.describe()

,avg_act_days,avg_days_ins,weekend_ratio,weeks_since_first_seen,wifi_ratio
count,61107.000000,7183.000000,63305.000000,67489.000000,63304.000000
mean,3.998519,5.228739,0.329158,4.877195,0.641166
std,2.115821,5.584633,0.270035,1.947677,0.398610
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,1.000000,0.129870,4.000000,0.221200
50%,4.000000,3.333333,0.279810,6.000000,0.842471
75%,6.000000,7.000000,0.458333,6.000000,1.000000
max,7.000000,28.000000,1.000000,6.000000,1.000000


In [93]:
part_0_v3 = part_0_v2.loc[user_df_v3.index, :]
part_0_v3.describe()

,buyer_d1,buyer_d7,buyer_d14,buyer_d28,buy_d7,buy_d14,buy_d28,iap_revenue_d7,iap_revenue_d14,iap_revenue_d28,...,retention_d3_to_d7,retention_d7_to_d14,retention_d1,retention_d3,retentiond7,avg_act_days,avg_days_ins,weekend_ratio,weeks_since_first_seen,wifi_ratio
count,67881.000000,67881.000000,67881.000000,67881.000000,67881.000000,67881.000000,67881.000000,67881.000000,67881.000000,67881.000000,...,60303.000000,60303.000000,60303.000000,60303.000000,60303.000000,61107.000000,7183.000000,63305.000000,67489.000000,63304.000000
mean,0.039510,0.051281,0.053638,0.055907,0.116321,0.149143,0.206287,13.878030,14.243944,15.123496,...,0.264431,0.214765,0.218828,0.153392,0.105799,3.998519,5.228739,0.329158,4.877195,0.641166
std,0.194807,0.220572,0.225304,0.229743,1.154502,1.546691,2.395827,2745.534599,2745.726939,2746.254657,...,0.441034,0.410663,0.413455,0.360368,0.307583,2.115821,5.584633,0.270035,1.947677,0.398610
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,0.129870,4.000000,0.221200
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,3.333333,0.279810,6.000000,0.842471
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,6.000000,7.000000,0.458333,6.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,136.000000,150.000000,179.000000,696622.140000,696622.140000,696622.140000,...,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000,28.000000,1.000000,6.000000,1.000000


In [95]:
part_0_v3

,buyer_d1,buyer_d7,buyer_d14,buyer_d28,buy_d7,buy_d14,buy_d28,iap_revenue_d7,iap_revenue_d14,iap_revenue_d28,...,rwd_prank,weekend_ratio,weeks_since_first_seen,wifi_ratio,whale_users_bundle_num_buys_prank,whale_users_bundle_revenue_prank,whale_users_bundle_total_num_buys,whale_users_bundle_total_revenue,row_id,datetime
0,0,1,1,1,1,1,1,2.147718,2.147718,2.147718,...,"[(25cdfd75fffab2e1953195cca3260a6f9d2b5d5d, 0....",0.019802,6.0,0.913366,None,None,None,None,819ecc0e-1a97-43ed-83f6-b9ede4f7fc48,2025-10-01-00-00
2,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[(221d7c94e54530510ec40b174a44cc60318dff55, 0....",0.399021,6.0,0.999388,None,None,None,None,fc1a2689-b136-4ffa-b23b-9d8215bd720f,2025-10-01-00-00
3,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[(3ba364431ad52210ba9b6286521264faecc93b34, 0....",0.121547,6.0,1.000000,None,None,None,None,0340fcc6-50bd-42ab-b9f4-4c1184b640cb,2025-10-01-00-00
4,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[(1031535cf2a1315422fd05d321349bcd3c3ffc04, 0....",0.293285,6.0,0.160243,None,None,None,None,219d253f-bef4-4039-84b2-ed55f009cc43,2025-10-01-00-00
6,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[(03ef279b474ca326883772428bbea95a28d788b8, 0....",0.125000,6.0,0.000000,None,None,None,None,f8ee7f14-94aa-411f-9bf5-964e8cc65803,2025-10-01-00-00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121879,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[(50d27130d9864e7e1078b533c957a10529899078, 0....",0.180190,6.0,0.907271,None,None,None,None,87718215-242c-4eb7-8267-e49c2d5da265,2025-10-01-00-00
121880,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[(8187185eaddc665fb56f52c1b3812192add643e9, 0....",0.324675,6.0,0.636364,None,None,None,None,62ecbb10-8026-4db4-b6f4-bf801f205d9e,2025-10-01-00-00
121883,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[(54662cebb1f386a180887921e549c9622d6896a6, 0.0)]",0.000000,6.0,0.285714,None,None,None,None,abc08919-7c5a-4a2d-8985-e51e90559c81,2025-10-01-00-00
121885,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[(3cab54b2b66a809e4ace91337d64ef0098949f47, 0.0)]",0.142857,2.0,0.095238,None,None,None,None,0452b7f5-e3e3-4f21-bdf1-9e17970c46f3,2025-10-01-00-00


### Replace NaN

#### By mean

In [99]:
part_0_v4 = part_0_v3.copy()

for col in ['avg_act_days', 'weekend_ratio', 'wifi_ratio']:
    part_0_v4.loc[:, col].fillna(part_0_v3[col].mean(), inplace = True)

In [100]:
part_0_v4.loc[:, ['avg_act_days', 'weekend_ratio', 'wifi_ratio']].isna().value_counts()

avg_act_days  weekend_ratio  wifi_ratio
False         False          False         67881
dtype: int64

#### By max

In [102]:
for col in ['avg_days_ins', 'weeks_since_first_seen']:
    part_0_v4.loc[:, col].fillna(part_0_v3[col].max(), inplace = True)

In [103]:
part_0_v4.loc[:, ['avg_days_ins', 'weeks_since_first_seen']].isna().value_counts()

avg_days_ins  weeks_since_first_seen
False         False                     67881
dtype: int64

#### Separation

In [ ]:
part_0_v5 = part_0_v4.join(part_0_v4["bcat"].apply(lambda x : dict(x)).apply(pd.Series))